In [1]:
import qml
import numpy as np

import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
import alchemy_tools as alch
import qml_interface as qi

In [2]:
# Import data
paths=qi.wrapper_alch_data()
alchemy_data, molecule_size = qi.load_alchemy_data(paths)


In [32]:
paths[4]

'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_000275/atomic_energies.txt'

In [3]:
# select training and validation set
training_set_size = 100
global_idc = qi.get_indices(len(alchemy_data), training_set_size)
local_idc = qi.get_local_idx(global_idc[0], molecule_size), qi.get_local_idx(global_idc[1], molecule_size)

# build representations and labels
local_reps = qi.generate_atomic_representations(alchemy_data, molecule_size)
local_labels = qi.generate_label_vector(alchemy_data, molecule_size.sum())

# optimize model
rep = local_reps[local_idc[0]], local_reps[local_idc[1]] # select the representations
labels = local_labels[local_idc[0]], local_labels[local_idc[1]] # select the labels
sigmas = np.logspace(-1, 4, 12) # list of hyperparameters for optimization
lams = np.logspace(-15, 0, 16)
opt_data, coeffs, errors = qi.optimize_hypar(rep, labels, sigmas, lams)

best_sigma = opt_data[np.where(opt_data[:,2]==np.amin(opt_data[:,2]))][0][0]
best_lambda = opt_data[np.where(opt_data[:,2]==np.amin(opt_data[:,2]))][0][1]

# The error in atomisation energy per molecule

In [17]:
# predicted atomic energergies
atomic_energies = qi.predict_labels(rep[1], rep[0], best_sigma, coeffs)
# true atomisation energies
ref_atomisation_en = np.empty(len(global_idc[1]))
for j,i in enumerate(global_idc[1]):
    ref_atomisation_en[j] = alchemy_data[i][:,6].sum()

# molecule size of molecules in test set
molecule_size_test = molecule_size[global_idc[1]]

# error per molecule
error_molecules = qi.calculate_error_atomisation_energy(atomic_energies, molecule_size_test, ref_atomisation_en)


In [47]:
# test calculate_error_atomisation_energy

# predict the energies for one molecule ('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_000275/atomic_energies.txt')
ae = qi.predict_labels(local_reps[qi.get_local_idx([global_idc[1][2]], molecule_size)], rep[0], best_sigma, coeffs)
# subtract the sum of the predicted energies from the sum of the correct atomic energies
np.abs(ae.sum()-alchemy_data[4][:,6].sum())

# compare value to the value in error molecules the third element for this random subset both values are the same
# indicating that the scheme works correctly

0.22224555062354234

In [48]:
error_molecules[0:4]

array([3.70468391, 3.8963704 , 0.22224555, 1.55482573])